<a href="https://colab.research.google.com/github/marcus-terra/geleia-grasp/blob/main/notebook/Grasp_Geleia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import random
import numpy  as np
import pandas as pd


### FUNCAO OBJETIVO (FITNESS)
### AS FORMAS DO CALCULO ESTAO DEFINIDAS NOS COMENTARIOS DENTRO DA FUNCAO
### É UMA FUNCAO DE MINIMIZACAO QUE RETORNA VALOR ENTRE 0 E 1 
def funcao_objetivo(solucao):
    
    violacoes = 0;
# A mesma sala não poderá ter mais de uma aula ao mesmo tempo;
# Essa restrição é atendida sempre devido a forma como a grade é montada

# 1) A mesma aula não poderá acontecer simultaneamente em salas diferentes;
# 2) Um professor não poderá dar 2 ou mais aulas ao mesmo tempo;
# A premissa 2) já engloba a premissa 1) pois cada disciplina
# tem sempre o mesmo professor
    total_dias = 5
    total_horarios = 2
    total_salas = 5
    for dia in range(total_dias):
        for horario in range(total_horarios):
            for sala_atual in range(total_salas):
                evento_atual = dia*total_dias*total_horarios+horario*total_salas+sala_atual 
                professor_atual = solucao[evento_atual][1]
                if (professor_atual != 'VAGO'):
                    for proxima_sala in range(sala_atual+1,total_salas):
                        proximo_evento = dia*total_dias*total_horarios+horario*total_salas+proxima_sala
                        proximo_professor = solucao[proximo_evento][1]
                        if (professor_atual == proximo_professor):
                            violacoes+=1
                            # evita a recontagem quando o mesmo professor 
                            # aparece mais de 2 vezes no mesmo dia e horario
                            break


# Caso a carga horária de uma aula seja maior que um 1 horário por semana,
# é desejável que as aulas não sejam na sequência imediata (mesmo dia);
# Como é uma premissa desejável e não obrigatória o peso da violacao é 0.1
    
    for dia in range(total_dias):
        for sala_h1 in range(total_salas):
            evento_h1 = dia*total_dias*total_horarios+sala_h1
            disciplina_h1 = solucao[evento_h1][0]
            if (disciplina_h1 != 'VAGO'):
                for sala_h2 in range(total_salas):
                    evento_h2 = dia*total_dias*total_horarios+sala_h2+total_salas
                    disciplina_h2 = solucao[evento_h2][0]
                    if (disciplina_h1 == disciplina_h2): # a mesma disciplina acontece no mesmo dia
                        if (sala_h1 == sala_h2): # se for na mesma sala a penalizacao reduz pela metade
                            violacoes+=0.05
                        else:
                            violacoes+=0.1
                        # a regra anterior já penaliza a mesma disciplina no mesmo horário
                        # assim para não penalizar novamente o laço é interrompido
                    break 
    

# É desejável que não existam buracos (horários/salas sem aula)
# na grade de horário
# Considera-se o número total de posicoes da grade (salas*dias*horarios) = 50 
# Como é uma premissa desejável e não obrigatória o peso da violacao é 0.1
    
    solucao_reversa = copy.deepcopy(solucao)
    solucao_reversa.reverse()
    buracos = 0;
    posicao_inicial = 0;
    while (posicao_inicial < 50 and solucao_reversa[posicao_inicial][1] == 'VAGO'):
        posicao_inicial+=1
    for i in range(posicao_inicial+1, len(solucao_reversa)):
        if (solucao_reversa[i][0] == 'VAGO'):
            buracos+=1
    violacoes = violacoes + 0.1*buracos;

# Solucao alternativa para as salas vagas (buracos na grade)
# Como não foi identificado problema de performance
# foi mantida a outra alternativa

    
#    ultimo_horario_ocupado = -1
#    salas_ocupadas = 0;
#    for i in range(0, len(solucao)):
#        if (solucao[i][0] != 'VAGO'):
#            ultimo_horario_ocupado = i
#            salas_ocupadas +=1
     # total de salas até o ultimo horario ocupado
#    total_de_salas = ultimo_horario_ocupado + 1
#    salas_vagas = total_de_salas - salas_ocupadas
#    violacoes = violacoes + 0.1*salas_vagas
 

# OBS: Uma questao importante é saber se o peso da violacao 
# de deixar um horario vago deve ser o mesmo que 
# colocar a mesma disciplina em sequencia

# O fitness (funcao objetivo) será :
# Melhor - quanto mais se aproximar de 0 (menos violacoes)
# Pior - quanto mais se aproximar de 1 (mais violacoes)
    custo = 1 - 1 / (1 * violacoes + 1);
    return custo


### GERA UMA SOLUCAO ALEATORIA [GRADE DE HORARIOS,FITNESS DA SOLUCAO] 
### COM BASE NO CONJUNTO DE AULAS NECESSARIAS 
### (DISCIPLINA, PROFESSOR)

def solucao_aleatoria(aulas):
    solucao = [[],float("inf")]
    # Solucao alternativa que gerar uma sequencia aleatorias de 50 numeros 
    # entre 0 e 49 que representam a ordem das aulas
    #sequencia = random.sample(list(range(0,len(aulas))), len(aulas))
    #solucao[0] = sequencia
    #solucao[1] = funcao_objetivo(aulas, sequencia)
    solucao[0] = random.sample(aulas, len(aulas))
    solucao[1] = funcao_objetivo(solucao[0])
    return solucao

### FUNCAO AUXILIAR DA "CALCULA_RCL" QUE DETERMINA O CUSTO DE SELEÇÃO DO
### PROXIMO PROFESSOR PARA A GRADE
### O CUSTO É BASEADO NA QUANTIDADE DE AULAS (HORARIOS) QUE UM PROFESSOR PRECISA
### MINISTRAR
### QUANTO MAIS AULAS -> MENOR O CUSTO PARA ESCOLHER ESSE PROFESSOR

def calcula_custos(candidatos):
    custos = []
    professores = []
    for i in range(0, len(candidatos)):
        if (candidatos[i][1] != 'VAGO'):
            professores.append(candidatos[i][1])
    for i in range(0, len(candidatos)):
        custo = 50 - professores.count(candidatos[i][1])
        custos.append(custo)
    return custos


### FUNCAO AUXILIAR DA "GRASP_CONSTRUCAO" QUE CONSTROI A LISTA RESTRITA DE
### CANDIDATOS - RCL (restricted candidate list) 
### A FUNCAO AVALIA OS CUSTOS DE CADA AULA CANDIDATA COM BASE NOS CUSTOS
### MÍNIMO E MÁXIMO ATUAIS E NO FATOR DE ALEATORIEDADE (ALFA_RCL)
### ALFA_RCL PRÓXIMO A ZERO -> BUSCA MAIS GULOSA
### ALFA_RCL PRÓXIMO A UM   -> BUSCA MAIS ALEATORIA 

def calcula_rcl(candidatos, custos, custo_min, custo_max, alfa_rcl):
    rcl = []
    for i in range(0, len(candidatos)):
        if (custos[i] <= custo_min + alfa_rcl*(custo_max - custo_min)):
            rcl.append(candidatos[i])
    return rcl

### FUNCAO DE CONSTRUCAO DE UMA SOLUCAO [GRADE DE HORARIOS,FITNESS DA SOLUCAO] 
### UTILIZANDO A META-HEURÍSTICA GRASP 
### (GREEDY RANDOMIZED ADAPTIVE SEARCH PROCEDURE)
### COM BASE NO CONJUNTO DE AULAS NECESSARIAS (DISCIPLINA, PROFESSOR)
### E NO FATOR DE ALEATORIEDADE DA RCL (ALFA_RCL)
### ALFA_RCL PRÓXIMO A ZERO -> BUSCA MAIS GULOSA
### ALFA_RCL PRÓXIMO A UM   -> BUSCA MAIS ALEATORIA 

def grasp_construcao(aulas, alfa_rcl):
    solucao = [[],float("inf")]
    candidatos = copy.deepcopy(aulas)
    custos = calcula_custos(candidatos)
    while (len(candidatos) > 0):
        custo_min = min(custos)
        custo_max = max(custos)
        rcl = calcula_rcl(candidatos, custos, custo_min, custo_max, alfa_rcl)
        elemento = rcl[random.randrange(len(rcl))]
        solucao[0].append(elemento)
        candidatos.remove(elemento)
        calcula_custos(candidatos)
    solucao[1] = funcao_objetivo(solucao[0])
    return solucao


### FUNCAO QUE, A PARTIR DE UMA SOLUCAO INICIAL, CALCULA UMA SOLUCAO VIZINHA
### O CALCULO É FEITO TROCANDO UMA ÚNICA POSICAO DA GRADE POR OUTRA DE FORMA 
### ALEATORIA (EX.: TROCA DO PRIMEIRO HORÁRIO DA SEGUNDA PELO ÚLTIMO DA SEXTA)

def calcula_vizinho(solucao_inicial):
    # Randomizacao apenas trocando as aulas no mesmo dia
    #horarios = random.sample(list(range(0,10)), 2)
    #sala = random.randrange(5)
    #trocas = [horarios[0]+10*sala, horarios[1]+10*sala]

    # Randomizacao trocando todas as posicoes na grade
    trocas = random.sample(list(range(0,len(solucao[0]))), 2)
    solucao = copy.deepcopy(solucao_inicial)
    auxiliar = solucao[0][trocas[0]]
    solucao[0][trocas[0]] = solucao[0][trocas[1]]
    solucao[0][trocas[1]] = auxiliar
    solucao[1] = funcao_objetivo(solucao[0])
    return solucao


### FUNCAO QUE A REALIZA UMA BUSCA LOCAL PELA MELHOR SOLUCAO 
### [GRADE DE HORARIOS,FITNESS DA SOLUCAO] A PARTIR DE UMA 
### SOLUCAO INICIAL. ESTA BUSCA LOCAL UTILIZA O ALGORITMO HILL CLIMBING.
### A QUANTIDADE PADRAO MÁXIMA DE SOLUCOES VIZINHAS CALCULADAS 
### PARA CADA NOVA MELHOR SOLUCAO ENCONTRADA É 1000 (MAX_ITERACOES)
### E O LIMITE PADRAO (THRESHOLD) PARA ENCERRAR A BUSCA LOCAL É ZERO

def busca_local(solucao_inicial, max_iteracoes = 1000, limite = 0):
    contador = 0
    melhor_solucao = copy.deepcopy(solucao_inicial)
    while (contador < max_iteracoes and melhor_solucao[1]>limite):
        solucao = calcula_vizinho(melhor_solucao)
        if (solucao[1] < melhor_solucao[1]):
            contador = 0
            melhor_solucao = copy.deepcopy(solucao)
        else: 
            contador = contador + 1 
    return melhor_solucao


### FUNCAO DE BUSCA DA MELHOR SOLUCAO GLOBAL [GRADE DE HORARIOS,FITNESS DA SOLUCAO] 
### UTILIZANDO A META-HEURÍSTICA GRASP 
### (GREEDY RANDOMIZED ADAPTIVE SEARCH PROCEDURE)
### COM BASE NO CONJUNTO DE AULAS NECESSARIAS (DISCIPLINA, PROFESSOR)
### NUMA SOLUCAO INICIAL (QUE PODE SER VAZIA)
### NUM NUMERO MÁXIMO DE ITERACOES DE BUSCA (MAX_ITERACOES - PADRAO = 5000)
### NO FATOR DE ALEATORIEDADE DA RCL (ALFA_RCL - PADRAO = 0.5) 
### ALFA_RCL PRÓXIMO A ZERO -> BUSCA MAIS GULOSA
### ALFA_RCL PRÓXIMO A UM   -> BUSCA MAIS ALEATORIA 
### E NO LIMITE (THRESHOLD) PARA ENCERRAR A BUSCA GLOBAL (LIMITE - PADRAO = 0)

def grasp(aulas, solucao_inicial = [[],float("inf")], max_iteracoes = 5000, alfa_rcl = 0.5, limite = 0):
    contador = 0
    melhor_solucao = copy.deepcopy(solucao_inicial)
    while (contador < max_iteracoes and melhor_solucao[1]>limite):
        solucao = grasp_construcao(aulas, alfa_rcl)
        solucao = busca_local(solucao, max_iteracoes = 100, limite = limite)
        if (solucao[1] < melhor_solucao[1]):
            melhor_solucao = copy.deepcopy(solucao) 
        contador = contador + 1
        print('Iteracao =', contador, '-> Custo Solucao =', melhor_solucao[1])
    return melhor_solucao


### FUNCAO QUE TRANSFORMA A ESTRUTURA DO TIPO
### [DISCIPLINA, PROFESSOR, QUANTIDADE DE HORARIOS EXIGIDO PELA DISCIPLINA] 
### EM UMA ESTRUTURA (GRADE) DE AULAS NECESSARIAS DO TIPO [DISCIPLINA, PROFESSOR]
### ONDE É REPLICADO O VALOR [DISCIPLINA, PROFESSOR] DE ACORDO COM A QUANTIDADE
### DE HORARIOS DE CADA DISCIPLINA EXIGE, COMO A GRADE COMPORTA ATÉ 50 HORÁRIOS 
### (NUMERO DE DIAS X NUMERO DE HORARIOS NO DIA X NUMERO DE SALAS), 
### CASO QUANTIDADE DE AULAS NECESSARIAS SEJA MENOR QUE O TAMANHO DA GRADE (50)
### O RESTANTE DA GRADE É PREENCHIDO COM VALORES ['VAGO','VAGO']

def gera_aulas(disc_prof_hor):
    # dt = np.dtype([('disciplina', np.unicode_, 4), ('professor', np.unicode_, 4)])
    aulas = [] 
    for i in range(0, disc_prof_hor.shape[0]):
        aula = disc_prof_hor[i] 
        num_periodos = aula[2]
        for j in range(0, num_periodos):
          aulas.append((aula[0],aula[1]))
    # Se a quantidade de aulas < 50 preenche a lista aulas com valores VAGO
    for i in range(len(aulas),50):
        aulas.append(('VAGO','VAGO'))
    return aulas #np.array(aulas,dt)

### FUNCAO QUE IMPRIME UMA SOLUCAO [GRADE DE HORARIOS, FITNESS DA SOLUCAO]
### DE FORMA AMIGÀVEL

def imprime_solucao(solucao):
    dias = ['Segunda','Terça','Quarta','Quinta','Sexta']
    for i in range(0,len(solucao[0])):
        dia = dias[i//10]
        if (i % 10 == 0):
            print('\n\n',dia)
        if (i % 5 == 0):
            print('\nHorario ', 1+ i % 2, '--------------------')
        print(solucao[0][i], end = ' | ')
    print('\nCusto da Solucao = ',solucao[1])    
    return

### METODO PRINCIPAL QUE RECEBE A URL DO CASO DE TESTE E RETORNA 
### A MELHOR SOLUCAO [GRADE DE HORARIOS, FITNESS DA SOLUCAO] ENCONTRADA
### 
def principal(url_caso_de_teste)
    disc_prof_hor = pd.read_csv(url_caso_de_teste, sep = ';')
    disc_prof_hor = disc_prof_hor.values
    aulas = gera_aulas(disc_prof_hor)
    solucao_inicial = solucao_aleatoria(aulas)
    melhor_solucao = grasp(aulas,solucao_inicial, max_iteracoes = 1000)
    imprime_solucao(melhor_solucao)
    return

principal('aulas03.csv')

FileNotFoundError: ignored